In [ ]:
!pip install geopandas

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as stats
import geopandas as gpd
from sklearn.preprocessing import KBinsDiscretizer
from google.colab import drive

np.random.seed(42)

In [ ]:

drive.mount('/content/drive')

In [ ]:
# Lectura del dataset
rainInAustralia = pd.read_csv('drive/MyDrive/Analisis de Datos - TP/weatherAUS.csv')  

# 2. **Análisis inicial**

## Visualización de primeras filas y resumen de cinco números

In [ ]:
rainInAustralia.head()

In [ ]:
rainInAustralia.tail()

In [ ]:
rainInAustralia.describe()

## Informacion del tipo de variables, valores faltantes, etc.



In [ ]:
rainInAustralia.info()

In [ ]:
# Porcentaje de datos faltantes
rainInAustralia.isnull().sum()/rainInAustralia.shape[0]*100

Observamos la presencia de varias columnas con más de 5% de valores faltantes, y con porcentajes tan elevados de hasta 48% (variable $\texttt{Sunshine}$).

### Análisis de variables numéricas

In [ ]:
def display_dataset_distributions(dataset):
    fig = dataset.hist(xlabelsize=12, ylabelsize=12,figsize=(15,7))
    [x.title.set_size(14) for x in fig.ravel()]
    plt.tight_layout()
    plt.show()

In [ ]:
# Histogramas de las variables numéricas (luego de eliminar los valores faltantes)
display_dataset_distributions(rainInAustralia)

### Chequeo de oblicuidad

In [ ]:
# Skewness
df_skew = rainInAustralia.skew()
print(df_skew)

Observamos variables con distribuciones cercanas a una normal ('MinTemp', 'Humidity3pm', 'Temp9am', entre otras), y otras con distribuciones fuertemente asimétricas ('Rainfall' y 'Evaporation'). Esta situacion puede obse|rvarse tambien en los histogramas graficados más arriba.

In [ ]:
def outlier_diagnostic_plots(df, variable):
    fig,axes = plt.subplots(1,3,figsize=(20,4))

    sns.histplot(df[variable], bins=30,ax=axes[0], kde=True)
    axes[0].set_title('Histograma')
    
    stats.probplot(df[variable], dist="norm", plot=axes[1])
    axes[1].set_title('QQ')
    
    # boxplot    
    sns.boxplot(y=df[variable],ax=axes[2])
    axes[2].set_title('Box&Whiskers')

In [ ]:
for col in rainInAustralia:
    if rainInAustralia[col].dtype.name == 'float64':
      if col=='Cloud9am' or col=='Cloud3pm':
        continue
      else:
        outlier_diagnostic_plots(rainInAustralia.dropna(subset = [col]), col)

In [ ]:
outlier_diagnostic_plots(rainInAustralia.dropna(subset = ['WindSpeed3pm']), 'WindSpeed3pm')

### Análisis de variables categóricas

In [ ]:
# Visualización de etiquetas diferentes para la variable 'Location'
unique_Location = rainInAustralia.Location.unique()
print(unique_Location)
print("Cantidad de etiquetas en la variable \'Location\':", len(unique_Location))

In [ ]:
# Visualización de etiquetas diferentes para las variables 'WindGustDir', 'WindDir9am' y 'WindDir3pm'
unique_WindDir = rainInAustralia.WindGustDir.unique()
print(unique_WindDir)
print("Cantidad de etiquetas en la variable \'WindGustDir\':", len(unique_WindDir))

In [ ]:
# Distribución de frencuencias de cada etiqueta
cat_cols = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'Cloud9am', 'Cloud3pm']
fig,axes = plt.subplots(len(cat_cols),1,figsize=(18,len(cat_cols)*4))
for i,col in enumerate(cat_cols):
    temp_df = pd.Series(rainInAustralia[col].value_counts() / len(rainInAustralia))
    temp_df.sort_values(ascending=False).plot.bar(ax=axes[i])
    axes[i].set_xlabel(col)
    #axes[i].axhline(y=0.05, color='red') # 5%
    axes[i].title.set_text(col)
    axes[i].set_ylabel('Frecuencia relativa')
    axes[i].set_xlabel('Categoría')
fig.tight_layout() 
#plt.show()

Se aprecia que la frecuencia relativo de las diferentes clases en las variables categóricas especificadas es bastante homogénea. Es decir que todas las etiquetas son importantes en la predicción.

In [ ]:
# Visualización de variables dicotómicas 'RainToday' y 'RainTomorrow'
fig, axs = plt.subplots(1, 2, figsize=(10, 10))
g = sns.countplot(x='RainToday', data=rainInAustralia, ax=axs[0])
g = sns.countplot(x='RainTomorrow', data=rainInAustralia, ax=axs[1])
plt.show()

### Analisis de multicolinealidad (Pearson) con el dataset original

In [ ]:
# Matriz de correlación, redondeo a 2 decimales
correlation_matrix = rainInAustralia.corr().round(2)
fig,axes = plt.subplots(1,1,figsize=(20,8))
sns.heatmap(data=correlation_matrix, annot=True,ax=axes);

Observamos algunas variables moderada a altamente correlacionadas ($|\rho|>=0.5$). La siguiente grafica muestra mas claramente dichas correlaciones máximas.

In [ ]:
dfCorr = rainInAustralia.corr()
filteredDf = dfCorr[((dfCorr >= .5) | (dfCorr <= -.5)) & (dfCorr !=1.000)]
plt.figure(figsize=(20,8))
sns.heatmap(filteredDf, annot=True)
plt.show()

### Valores extremos

In [ ]:
def outlier_diagnostic_plots(df, variable):
    fig,axes = plt.subplots(1,3,figsize=(20,4))

    sns.histplot(df[variable], bins=30,ax=axes[0], kde=True)
    axes[0].set_title('Histograma')
    
    stats.probplot(df[variable], dist="norm", plot=axes[1])
    axes[1].set_title('QQ')
    
    # boxplot    
    sns.boxplot(y=df[variable],ax=axes[2])
    axes[2].set_title('Box&Whiskers')

In [ ]:
outlier_diagnostic_plots(rainInAustralia, 'Humidity9am')

In [ ]:
data = np.asarray(rainInAustralia.Humidity9am.dropna())
q25, q75 = np.percentile(data, 25), np.percentile(data, 75)
iqr = q75 - q25
cutoff = iqr * 1.5
lower, upper = q25-cutoff, q75+cutoff
outliers_idx = np.argwhere((data<lower) | (data>upper))

In [ ]:
fig,axes = plt.subplots(1,figsize=(18,6))
axes.scatter(np.arange(0,data.shape[0]),data,color='g')
axes.scatter(outliers_idx,data[outliers_idx],color='r')
axes.axhline(lower,color="r");axes.axhline(upper,color="r");

In [ ]:
def find_skewed_boundaries(df, variable, distance=1.5):
    IQR = df[variable].quantile(0.75) - df[variable].quantile(0.25)
    lower_boundary = df[variable].quantile(0.25) - (IQR * distance)
    upper_boundary = df[variable].quantile(0.75) + (IQR * distance)
    return upper_boundary, lower_boundary

In [ ]:
Rf_upper_limit, Rf_lower_limit = find_skewed_boundaries(rainInAustralia, 'Rainfall', 1.5)

In [ ]:
outliers_Rf = np.where(rainInAustralia['Rainfall'] > Rf_upper_limit, True,
                       np.where(rainInAustralia['Rainfall'] < Rf_lower_limit, True, False))

In [ ]:
rainInAustralia_trimmed = rainInAustralia.loc[~(outliers_Rf), ]
rainInAustralia.shape, rainInAustralia_trimmed.shape

In [ ]:
rainInAustralia_trimmed.shape[0]/rainInAustralia.shape[0]

In [ ]:
outlier_diagnostic_plots(rainInAustralia_trimmed, 'Rainfall')

---
# Ingeniera de features

In [ ]:
percetangeOfMissing=rainInAustralia.isnull().sum()/rainInAustralia.shape[0]*100
columnsToDelete=percetangeOfMissing[percetangeOfMissing>30]
columnsToDelete=list(columnsToDelete.keys())
columnsToDelete

In [ ]:
featuredDS=rainInAustralia.copy()
featuredDS=featuredDS.drop(columns=columnsToDelete)

In [ ]:
featuredDS.head()

In [ ]:
featuredDS 


## Relacion estacional entre fechas y humedad

In [ ]:
years=list(range(2009,2017))
yearsSTR=[str(x) for x in years]
fig,axes = plt.subplots(len(yearsSTR),1,figsize=(18,len(yearsSTR)*4))
print(yearsSTR)

for i,year in enumerate(yearsSTR):
    yearDS=featuredDS[featuredDS['Date'].str.contains(year)]
    yearDS=yearDS[featuredDS['Location'].str.contains("Albury")]
    axes[i].plot(range(0,len(yearDS.Humidity3pm)),yearDS.Humidity3pm)
    #axes[i].plot(yearDS.Date,yearDS.Temp3pm)
    axes[i].title.set_text("Humedad "+str(year))
    axes[i].set_ylabel('Frecuencia relativa')
    axes[i].set_xlabel('Categoría')
    



## Relacion estacional entre fechas y temperaturas

In [ ]:
years=list(range(2009,2017))
yearsSTR=[str(x) for x in years]
fig,axes = plt.subplots(len(yearsSTR),1,figsize=(18,len(yearsSTR)*4))
print(yearsSTR)

for i,year in enumerate(yearsSTR):
    yearDS=featuredDS[featuredDS['Date'].str.contains(year)]
    yearDS=yearDS[featuredDS['Location'].str.contains("Albury")]
    axes[i].plot(range(0,len(yearDS.Temp3pm)),yearDS.Temp3pm)
    #axes[i].plot(yearDS.Date,yearDS.Temp3pm)
    axes[i].title.set_text("Temperatura "+str(year))
    axes[i].set_ylabel('Frecuencia relativa')
    axes[i].set_xlabel('Categoría')
    



## Relacion estacional entre fechas y presion

In [ ]:
years=list(range(2009,2017))
yearsSTR=[str(x) for x in years]
fig,axes = plt.subplots(len(yearsSTR),1,figsize=(18,len(yearsSTR)*4))
print(yearsSTR)

for i,year in enumerate(yearsSTR):
    yearDS=featuredDS[featuredDS['Date'].str.contains(year)]
    yearDS=yearDS[featuredDS['Location'].str.contains("Albury")]
    axes[i].plot(range(0,len(yearDS.Pressure3pm)),yearDS.Pressure3pm)
    #axes[i].plot(yearDS.Date,yearDS.Temp3pm)
    axes[i].title.set_text("Presion "+str(year))
    axes[i].set_ylabel('Frecuencia relativa')
    axes[i].set_xlabel('Categoría')
    


## Discretizacion variables de direccion de viento

In [ ]:
np.unique(featuredDS.Location)

In [ ]:
np.unique(featuredDS.WindDir9am.dropna())

In [ ]:
windDirection={}
windDirection["E"]=[1,0]
windDirection["ENE"]=[1,0]

In [ ]:
windDirection={}
xs=[]
ys=[]
for e in np.unique(featuredDS.WindDir9am.dropna()):
  x,y=0,0
  for i in e:
    if("E" in i):
      x=x+1
    if("W" in i):
      x=x-1
    if("N" in i):
      y=y+1
    if("S" in i):
      y=y-1
  

  numer=(x**2+y**2)**(1/2)
  print(e,x/numer,y/numer)
  xs.append(x/numer)
  ys.append(y/numer)
  windDirection[e]=[x/numer,y/numer]



In [ ]:
windDirection
xs.append(0)
ys.append(0)
plt.scatter(xs,ys)

plt.rcParams["figure.figsize"] = (10,10)
plt.axes().set_aspect('equal')

for i, txt in enumerate(windDirection.keys()):
    plt.annotate(txt, (xs[i], ys[i]),fontsize=25)

## Discretización de variable Rainfall

In [ ]:
from logging import exception
qOfBins=5
data=np.array(featuredDS.Rainfall.dropna()[(featuredDS.Rainfall.dropna()<300) & (featuredDS.Rainfall!=0)])

kbins = KBinsDiscretizer(n_bins=qOfBins, encode= 'ordinal' , strategy= 'quantile' )
data_tf = kbins.fit_transform(data.reshape(-1,1))
fig,axes = plt.subplots(2,figsize=(18,8))
axes[0].hist(data_tf, bins=qOfBins);
axes[1].scatter(np.arange(0,data.shape[0]),data)
[axes[1].axhline(x,color='r') for x in kbins.bin_edges_[0]];
print(kbins.bin_edges_)
#print(kbins.strategy)

In [ ]:
data=np.array(rainInAustralia.Rainfall)


data_tf = data.reshape(-1,1)
plt.rcParams["figure.figsize"] = (20,4)
plt.scatter(np.arange(0,data.shape[0]),data)


In [ ]:
np.sort(featuredDS.Rainfall.values)

In [ ]:
featuredDS=featuredDS[(featuredDS.Rainfall<300)]
featuredDS["RainfallEncoded"]=featuredDS.Rainfall.copy()
featuredDS.RainfallEncoded.astype(str).astype(float)

In [ ]:
for i in range(len(kbins.bin_edges_[0])-1):
  print(kbins.bin_edges_[0][i],kbins.bin_edges_[0][i+1])
  lower=kbins.bin_edges_[0][i]
  upper=kbins.bin_edges_[0][i+1]
  if(i<len(kbins.bin_edges_[0])):
    upper+=0.0001
  print(len(featuredDS.RainfallEncoded[(featuredDS.RainfallEncoded.astype(float)<upper) & (featuredDS.RainfallEncoded.astype(float)>=lower)]))
  featuredDS.RainfallEncoded[(featuredDS.RainfallEncoded.astype(float)<upper) & (featuredDS.RainfallEncoded.astype(float)>=lower)]=str(i+5000)


print("0",len(featuredDS.RainfallEncoded[featuredDS.RainfallEncoded==0]))
#featuredDS.RainfallEncoded[featuredDS.RainfallEncoded.astype(float)==0]=str(0+5000)

In [ ]:
for i,data in enumerate(np.unique(list(featuredDS['RainfallEncoded'].value_counts().index))):
  featuredDS.RainfallEncoded[featuredDS.RainfallEncoded==data]=i


featuredDS.RainfallEncoded=featuredDS.RainfallEncoded.astype(int)

In [ ]:
np.unique(list(featuredDS['RainfallEncoded'].value_counts().index))

In [ ]:
data_tf2 = kbins.transform(np.array(featuredDS.Rainfall).reshape(-1,1))
fig,axes = plt.subplots(1,figsize=(12,8))
sns.countplot(data=featuredDS,x="RainfallEncoded") #,order = featuredDS['RainfallEncoded'].value_counts().index
#axes[1].scatter(np.arange(0,featuredDS.Rainfall.shape[0]),featuredDS.Rainfall)
#[axes[1].axhline(x,color='r') for x in kbins.bin_edges_[0]];
print(kbins.bin_edges_)
print(data_tf2.flatten())

In [ ]:
featuredDS.drop(columns=["Rainfall"],inplace=True)

In [ ]:
featuredDS.RainfallEncoded.unique()

## Removemos registros donde haya Nans en la variable objetivo

In [ ]:
#np.unique()
import datetime
import math
experimentationTest=featuredDS[featuredDS.RainTomorrow.isna()]

locations=experimentationTest.Location.unique()
for row in experimentationTest.iterrows():
  #print(row[1].Location)
  currentDS=featuredDS[featuredDS.Location==row[1].Location]
  date=datetime.datetime.strptime(row[1].Date,"%Y-%m-%d")
  #print(date+datetime.timedelta(days=1))
  nextDay=date+datetime.timedelta(days=1)
  #
  nextDayValue=currentDS[currentDS.Date==nextDay.strftime("%Y-%m-%d")].RainToday
  try:
    if(len(nextDayValue.values)!=0 and math.isnan(nextDayValue.values[0])==False):
      print(row[1].Location,row[1].Date,currentDS[currentDS.Date==nextDay.strftime("%Y-%m-%d")].RainToday.values[0])
      print("-----------------------------")
    else:
      if(len(nextDayValue.values)==0):
        print("Do not exist in DS ",nextDay,row[1].Location)
  except:
    print(nextDay,row[1].Location)
  #print(currentDS[currentDS.Date==date.strftime("%Y-%m-%d")].RainTomorrow.values)
  #print()


  

In [ ]:
#np.unique()
experimentationTest=featuredDS[featuredDS.RainToday.isna()]
locations=experimentationTest.Location.unique()
print(len(experimentationTest))
for row in experimentationTest.iterrows():
  #print(row[1].Location)
  currentDS=featuredDS[featuredDS.Location==row[1].Location]
  date=datetime.datetime.strptime(row[1].Date,"%Y-%m-%d")
  #print(date+datetime.timedelta(days=1))
  priorDay=date+datetime.timedelta(days=-1)
  #
  priorDayValue=currentDS[currentDS.Date==priorDay.strftime("%Y-%m-%d")].RainTomorrow
  try:
    if(len(priorDayValue.values)!=0 and math.isnan(priorDayValue.values[0])==False):
      print(row[1].Location,row[1].Date,currentDS[currentDS.Date==priorDay.strftime("%Y-%m-%d")].RainTomorrow.values[0])
      print("-----------------------------")
    else:
      if(len(priorDayValue.values)==0):
        print("Do not exist in DS ",priorDay,row[1].Location)
  except:
    print(priorDay,row[1].Location)
    
  #print(currentDS[currentDS.Date==date.strftime("%Y-%m-%d")].RainTomorrow.values)
  #print()


  

In [ ]:
#Removemos rows con variable objetivo NaN
experimentationTest=featuredDS[featuredDS.RainTomorrow.isna()]
rowsWithNoRainTomorrow=np.array(experimentationTest.index)
featuredDS.drop(rowsWithNoRainTomorrow,axis=0,inplace=True)

## Codificacion de variables categóricas binarias

In [ ]:
#Codificamos los Rain
featuredDS.RainToday=featuredDS.RainToday.replace("No",int(0)).replace("Yes",int(1))
featuredDS.RainTomorrow=featuredDS.RainTomorrow.replace("No",int(0)).replace("Yes",int(1))

In [ ]:
featuredDS.RainToday=np.array(featuredDS.RainToday,dtype=int)
featuredDS.RainTomorrow=np.array(featuredDS.RainTomorrow,dtype=int)

In [ ]:
#Codificamos los wind
windAdresses=["WindGustDir","WindDir9am","WindDir3pm"]

In [ ]:

for windAdd in windAdresses:
  for id,i in enumerate(["X","Y"]):
    dfDir=featuredDS[windAdd]
    for windDir in windDirection.keys():      
      dfDir=dfDir.replace(windDir,windDirection[windDir][id])
    #print(dfDir)
    featuredDS[windAdd+i]=dfDir


In [ ]:
featuredDS=featuredDS.drop(columns=windAdresses)
featuredDS

## Codificación de variable "Location"

In [ ]:
countries = gpd.read_file(
               gpd.datasets.get_path("naturalearth_lowres"))
countries.head()

In [ ]:
df = pd.read_csv("drive/MyDrive/Analisis de Datos - TP/au.csv", 
                 usecols=["city", "lat", "lng"])
df = df.rename(columns={'city': 'Location'})
df.head()

if df.iloc[452].Location=="Perth":
  df=df.drop(452)


In [ ]:
featuredDS2=featuredDS.copy()
newNames={}
for citi in np.unique(featuredDS.Location):
  firstComponent=""
  secondComponent=""
  for id,letter in enumerate(citi):
    if id != 0 and id != 1:
      if (letter.isupper()):
        firstComponent=citi[0:id]
        secondComponent=citi[id:len(citi)]
        break
  
  #print(firstComponent,secondComponent)
  if(firstComponent!=""):
    newNames[citi]=firstComponent+" "+secondComponent

for citi in newNames.keys():
  featuredDS2.Location[featuredDS.Location==citi]=newNames[citi]

In [ ]:
#featuredDS2.Location[featuredDS2.Location=="Perth Airport"]="Perth"
#featuredDS2.Location[featuredDS2.Location=="Sydney Airport"]="Sydney"
#featuredDS2.Location[featuredDS2.Location==newNames["MelbourneAirport"]]="Melbourne"
#featuredDS2.Location[featuredDS2.Location=="Melbourne Airport"]="Melbourne"


In [ ]:
import time
from  geopy.geocoders import Nominatim
newLocs={}
noLoc=[]
geolocator = Nominatim()
for citi in np.unique(featuredDS2.Location):
  if citi not in np.unique(df.Location):
    time.sleep(1) 
    print(citi)
    country ="Australia"
    try:
      loc = geolocator.geocode(citi+','+ country)
      print("latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)
      newLocs[citi]=[loc.latitude,loc.longitude]
    except:
      print("No loc for ",citi)
      noLoc.append(citi)


noLoc

In [ ]:
df2=df.copy()
for city in  newLocs.keys():
  loc=[city,newLocs[city][0],newLocs[city][1]]
  df2 = df2.append(pd.Series(loc, index = ["Location","lat","lng"]), 
           ignore_index=True)

df2

In [ ]:
citiesInfo = pd.merge(df2, featuredDS2, how='inner', on=['Location']) #.drop_duplicates()

In [ ]:
citiesInfo=citiesInfo[['Location','lat','lng']]

In [ ]:
citiesInfo.value_counts()

In [ ]:
# initialize an axis
fig, ax = plt.subplots(figsize=(25,20))
qOfCitiesToShow=45
# plot map on axis
countries = gpd.read_file(  
     gpd.datasets.get_path("naturalearth_lowres"))
countries[countries["name"] == "Australia"].plot(color="lightgrey",
                                                 ax=ax)

# plot points
citiesInfo.drop_duplicates().head(qOfCitiesToShow).plot.scatter(x="lng", y="lat",c=citiesInfo.value_counts().head(qOfCitiesToShow), #kind="scatter", 
       colormap="hsv", colorbar=True,
        title="Location Of Cities", 
        ax=ax)
# add grid
ax.grid(b=True, alpha=0.5)

for idx, row in citiesInfo.drop_duplicates().head(qOfCitiesToShow).iterrows():
    ax.annotate(row['Location'], (row['lng'], row['lat']) )

# get axes limits
x_lo, x_up = ax.get_xlim()
y_lo, y_up = ax.get_ylim()
# add minor ticks with a specified sapcing (deg)
deg = 1
ax.set_xticks(np.arange(np.ceil(x_lo), np.ceil(x_up), deg), minor=True)
ax.set_yticks(np.arange(np.ceil(y_lo), np.ceil(y_up), deg), minor=True)
ax.grid(b=True, which="minor", alpha=0.25)

plt.show()

In [ ]:
featuredDS2=pd.merge(citiesInfo.drop_duplicates(), featuredDS2, how='inner', on=['Location']).drop_duplicates()


In [ ]:
for locationn in np.unique(featuredDS2.Location):
  if(len(featuredDS2[featuredDS2.Location==locationn])!=len(np.unique(featuredDS2[featuredDS2.Location==locationn].Date))):
    print(locationn,len(featuredDS2[featuredDS2.Location==locationn]),len(np.unique(featuredDS2[featuredDS2.Location==locationn].Date)))

featuredDS2=featuredDS2.drop(columns=["Location"])

In [ ]:
years=list(range(2009,2017))
yearsSTR=[str(x) for x in years]
fig,axes = plt.subplots(len(yearsSTR),1,figsize=(18,len(yearsSTR)*4))
print(yearsSTR)

for i,year in enumerate(yearsSTR):
    #-36.080600  146.915800
    yearDS=featuredDS2[featuredDS2['Date'].str.contains(year)]
    yearDS=yearDS[featuredDS2['lat']==-36.080600]
    yearDS=yearDS[featuredDS2['lng']==146.915800]
    axes[i].scatter(range(0,len(yearDS.RainTomorrow)),yearDS.RainTomorrow)
    #axes[i].plot(yearDS.Date,yearDS.Temp3pm)
    axes[i].title.set_text(year)
    axes[i].set_ylabel('Frecuencia relativa')
    axes[i].set_xlabel('Categoría')
    


## Codificación de variable "Date" Con fases lunares(no se hizo)

---





In [ ]:

import math, decimal, datetime
dec = decimal.Decimal

def position(now=None): 
   if now is None: 
      now = datetime.datetime.now()

   diff = now - datetime.datetime(2001, 1, 1)
   days = dec(diff.days) + (dec(diff.seconds) / dec(86400))
   lunations = dec("0.20439731") + (days * dec("0.03386319269"))

   return lunations % dec(1)

def phase(pos): 
   index = (pos * dec(8)) + dec("0.5")
   index = math.floor(index)
   return {
      0: "New Moon", 
      1: "Waxing Crescent", 
      2: "First Quarter", 
      3: "Waxing Gibbous", 
      4: "Full Moon", 
      5: "Waning Gibbous", 
      6: "Last Quarter", 
      7: "Waning Crescent"
   }[int(index) & 7]

def main(): 
   pos = position()
   phasename = phase(pos)

   roundedpos = round(float(pos), 3)
   print("%s (%s)" % (phasename, roundedpos))

In [ ]:
float(position(datetime.datetime(2022,6,14)))

In [ ]:
yearDS=featuredDS2[featuredDS2['Date'].str.contains("2011")]
#np.unique()
np.abs(featuredDS2.RainTomorrow.values)
np.abs([0,1,-1])
rainInAustralia.RainTomorrow.isna().sum()

## Codificación variable "Date"

In [ ]:
#datesInMonth=featuredDS2[featuredDS2.Date.str.contains("2012-06")].Date[0:30].values
#date=[datetime.datetime.strptime(dat,"%Y-%m-%d") for dat in datesInMonth]
quantityOfDays=31-0
deegreForDays=360/quantityOfDays
angleForEveryDay=2*math.pi/quantityOfDays
angleForEveryMonth=2*math.pi/12
Xs=[]
Ys=[]
for i,day in enumerate(range(12)):
  Xs.append(math.sin(i*angleForEveryMonth))
  Ys.append(math.cos(i*angleForEveryMonth))
#math.sin(math.pi),math.cos(math.pi)
months=["ENE","FEB","MAR","ABR","MAY","JUN","JUL","AGO","SEP","OCT","NOV","DIC"]
Xs.append(0)
Ys.append(0)
plt.scatter(Xs,Ys)
plt.rcParams["figure.figsize"] = (10,10)
plt.axes().set_aspect('equal')

for i, txt in enumerate(months):
    plt.annotate(txt, (Xs[i], Ys[i]),fontsize=25)

In [ ]:
Xs=[]
Ys=[]
for i,day in enumerate(range(1,32)):
  Xs.append(math.sin(i*angleForEveryDay))
  Ys.append(math.cos(i*angleForEveryDay))
#math.sin(math.pi),math.cos(math.pi)
days=range(1,32)
Xs.append(0)
Ys.append(0)
plt.scatter(Xs,Ys)
plt.rcParams["figure.figsize"] = (10,10)
plt.axes().set_aspect('equal')


for i, txt in enumerate(days):
    plt.annotate(txt, (Xs[i], Ys[i]),fontsize=25)

In [ ]:
featuredDS2['Date'] = pd.to_datetime(featuredDS2['Date'], format="%Y-%m-%d")
featuredDS2['Date'] 

In [ ]:
featuredDS2['day'] = featuredDS2['Date'].dt.day
featuredDS2['month'] = featuredDS2['Date'].dt.month
featuredDS2['year'] = featuredDS2['Date'].dt.year
featuredDS2.drop(columns=["Date"],inplace=True)

In [ ]:
np.sin([math.pi,2,3])
featuredDS2['dayXEncoded']=np.round(np.sin(angleForEveryDay*featuredDS2['day'].values),10)
featuredDS2['dayYEncoded']=np.round(np.cos(angleForEveryDay*featuredDS2['day'].values),10)

In [ ]:
featuredDS2['monthXEncoded']=np.round(np.sin(angleForEveryMonth*featuredDS2['month'].values),10)
featuredDS2['monthYEncoded']=np.round(np.cos(angleForEveryMonth*featuredDS2['month'].values),10)


In [ ]:
featuredDS2.drop(columns=["day","month"],inplace=True)

In [ ]:
featuredDS2.head()

## Correlación de Pearson previo a imputacion

In [ ]:
correlation_matrix = featuredDS2.corr().round(2)
fig,axes = plt.subplots(1,1,figsize=(20,8))
sns.heatmap(data=correlation_matrix, annot=True,ax=axes);

##Imputacion

In [ ]:
categoricVars=["RainToday","RainfallEncoded"]#Rainfall
numericVars=featuredDS2.columns.values

In [ ]:
numericVars=numericVars[numericVars!="RainToday"]
numericVars=numericVars[numericVars!="RainfallEncoded"]
numericVars=numericVars[numericVars!="RainTomorrow"]


In [ ]:
featuredDS2.info()

In [ ]:
import matplotlib.colors as mcolors

a=featuredDS2.isnull().sum()/featuredDS2.shape[0]*100
a=a[a>0]
print(a.index)
print(a.values)
plt.rcParams["figure.figsize"] = (26,6)
plt.bar(a.index,a.values,color=mcolors.BASE_COLORS)

plt.title("Percentage of NaNs")


In [ ]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import Ridge

ridge = Ridge()
logisticR = LogisticRegression()

imp = IterativeImputer(n_nearest_features=4,estimator=logisticR,missing_values=np.nan, max_iter=20, verbose=2, imputation_order='roman',random_state=0)
featuredDS2[categoricVars]=imp.fit_transform(featuredDS2[categoricVars])
#featuredDS2[featuredDS2.columns[~featuredDS2.columns.isin(categoricVars)]]=imp.fit_transform(featuredDS2[featuredDS2.columns[~featuredDS2.columns.isin(categoricVars)]])

In [ ]:

imp = IterativeImputer(n_nearest_features=4,estimator=ridge,missing_values=np.nan, max_iter=20, verbose=2, imputation_order='roman',random_state=0)
featuredDS2[numericVars]=imp.fit_transform(featuredDS2[numericVars])

In [ ]:
numericVars

## Correlación de Pearson

In [ ]:
correlation_matrix = featuredDS2[numericVars].corr().round(2)
filteredDf = correlation_matrix[((correlation_matrix >= .5) | (correlation_matrix <= -.5)) & (correlation_matrix !=1.000)]
fig,axes = plt.subplots(1,1,figsize=(20,8))
sns.heatmap(data=filteredDf.dropna(axis=0, how='all', thresh=None, subset=None, inplace=False).dropna(axis=1, how='all', thresh=None, subset=None, inplace=False), annot=True,ax=axes);

In [ ]:
s = filteredDf.abs().unstack() #filteredDf
so = s.sort_values(kind="quicksort", ascending=False).drop_duplicates()
vars=so.loc[(so<1) & (so>0.79), :] 
print(vars)

## Correlacion de Spearman

In [ ]:
correlation_matrix = featuredDS2[numericVars].corr("spearman").round(2)
filteredDf = correlation_matrix[((correlation_matrix >= .5) | (correlation_matrix <= -.5)) & (correlation_matrix !=1.000)]
fig,axes = plt.subplots(1,1,figsize=(20,8))
sns.heatmap(data=filteredDf.dropna(axis=0, how='all', thresh=None, subset=None, inplace=False).dropna(axis=1, how='all', thresh=None, subset=None, inplace=False), annot=True,ax=axes);

In [ ]:
s = filteredDf.abs().unstack() #filteredDf
so = s.sort_values(kind="quicksort", ascending=False).drop_duplicates()
vars=so.loc[(so<1) & (so>0.79), :] 
print(vars)

## Correlación de Kendall

In [ ]:
customVars=numericVars.copy()
customVars=customVars[customVars!="Rainfall"]
customVars=np.append(customVars,"RainTomorrow")
print(customVars)
correlation_matrix = featuredDS2[customVars].corr("kendall").round(2)
fig,axes = plt.subplots(1,1,figsize=(4,8))
sns.heatmap(data=correlation_matrix[["RainTomorrow"]], annot=True,ax=axes);

## Drop de columnas con alta correlacion

In [ ]:
listOfColumnsWithHighCorrelation=[]
for key in vars.keys():
  listOfColumnsWithHighCorrelation.append(key[0])

print(listOfColumnsWithHighCorrelation)

In [ ]:
featuredDS2org=featuredDS2.copy()
featuredDS2=featuredDS2.drop(columns=listOfColumnsWithHighCorrelation)

## Prueba de transformación de Yeo Johnson para variables con distribución no-normal

In [ ]:
from sklearn.preprocessing import PowerTransformer, MinMaxScaler
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import QuantileTransformer

In [ ]:
#Check Yeo Johnson to remove skewness and kurtosis
from scipy.stats import powerlognorm
weirdVars=["Humidity9am","Humidity3pm","WindGustSpeed","WindSpeed9am","WindSpeed3pm"]

fig = plt.figure(figsize=(12, 15), facecolor="white")

for i, wV in enumerate(weirdVars):
  
  ax1 = fig.add_subplot(5, 2, i*2+1)
  x = featuredDS2[wV].values
  prob = stats.probplot(x, dist=stats.norm, plot=ax1)
  ax1.set_xlabel('')
  ax1.set_title('Probplot of '+wV+' against normal distribution')

  ax2 = fig.add_subplot(5, 2, (i+1)*2)
  xt, lmbda = stats.yeojohnson(x)
  prob = stats.probplot(xt, dist=stats.norm, plot=ax2)
  ax2.set_xlabel('')
  ax2.set_title('Probplot of '+wV+' after Y-J transformation')

fig.tight_layout()

In [ ]:
featuredDS2.info()

---
---
# **MODELOS**

##Division de datos

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X=featuredDS2.drop(columns="RainTomorrow")
y=featuredDS2["RainTomorrow"]

scaler = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

scaler.fit(X_train)

X_train=scaler.transform(X_train)

X_test=scaler.transform(X_test)



## Balanceo de clases

In [ ]:
import imblearn
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)


## Regresión logística

In [ ]:
#Regresion Logistica
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

y_pred=clf.predict(X_test)

print(classification_report(y_test, y_pred))


## Arboles de decisión

In [ ]:
#DecisionTree
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)
y_pred=clf.predict(X_test)

print(classification_report(y_test, y_pred))

## XGBoost

In [ ]:
#XGBoost
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(X_train, y_train)
y_pred=model.predict(X_test)

print(classification_report(y_test, y_pred))


## Random forest

In [ ]:
#RandomForest
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100,max_depth=16, random_state=0,n_jobs=-1).fit(X_train, y_train)
y_pred=clf.predict(X_test)
print(classification_report(y_test, y_pred))



## MLP

In [ ]:
#MLP
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1, max_iter=3000,early_stopping=True,verbose=False,solver='sgd').fit(X_train, y_train)
y_pred=clf.predict(X_test)
print(classification_report(y_test, y_pred))


## Grafico de metricas

In [ ]:
models=["RL","DT","XGB","RF","MLP"]
avgACCU=np.array([0.79,0.53,0.83,0.85,0.81])
macroF1Score=np.array([0.79,0.63,0.76,0.78,0.76])
precision=np.array([0.83,0.79,0.84,0.85,0.84])
cl1Recall=np.array([0.76,0.58,0.66,0.67,0.77])

  
X_axis = np.arange(len(models))
  
plt.bar(X_axis - 0.2, avgACCU*100, 0.2, label = 'AVG Accuracy')
plt.bar(X_axis , macroF1Score*100, 0.2, label = 'F1 Score')
plt.bar(X_axis + 0.2, precision*100, 0.2, label = 'Precision')
plt.bar(X_axis + 0.4, cl1Recall*100, 0.2, label = 'Recall(rain)')

axes = plt.gca()
axes.yaxis.label.set_size(16)

plt.xticks(X_axis, models,fontsize=15)
plt.yticks(fontsize=15)

#plt.xlabel("Modelos",fontsize=30)

#plt.title("Metricas",fontsize=30)

plt.rcParams["figure.figsize"] = (30,10)

plt.legend(fontsize=15)
plt.show()
